In [1]:
import random
import smtplib
import yaml

In [2]:
def read_config(config_file):
    '''
    Reads the elements of a YAML configuration file.
    
    Args:
    config_file: filename (and path) to configuration file
    
    Returns:
    dictionary of configuration file enteries
    '''
    
    with open(config_file,"r") as file:
        cfg = yaml.safe_load(file)
        
    return cfg

In [3]:
def dict_key_to_list(dictionary):
    '''
    Creates list from hierarchal/multi-tiered dictionary keys. 
    Primarily intended for dictionaries generated by the 'read_config'
    function for people names.
    
    Args:
    dictionary: dictionary
    
    Returns:
    list of key values
    '''
    
    # Empty list
    dictlist = []
    
    for key,value in dictionary.items():
        temp = key
        dictlist.append(temp)

    return dictlist 

In [4]:
cfg_file = "config.gift.yml"
ppl_file = "config.people.yml"

In [8]:
auto = read_config(cfg_file); auto

{'email': 'throwaway.account@mail.com', 'password': 'password'}

In [7]:
ppl_dict = read_config(ppl_file); ppl_dict

{'Person1': {'email': 'Person1@mail.com', 'exclude': 'Person2'},
 'Person2': {'email': 'Person2@mail.com'},
 'Person3': {'email': 'Person3@mail.com'},
 'Person4': {'email': 'Person4@mail.com'}}

In [9]:
new_dict = {}

In [10]:
new_dict['Person1'] = "something"

In [11]:
new_dict

{'Person1': 'something'}

In [15]:
ppl_dict.keys()

dict_keys(['Person1', 'Person2', 'Person3', 'Person4'])

In [16]:
for p in ppl_dict.keys():
    print(p)

Person1
Person2
Person3
Person4


In [32]:
new_dict = dict(ppl_dict.keys()); new_dict

ValueError: dictionary update sequence element #0 has length 7; 2 is required

In [29]:
new_dict.pop("Person1")

'something2'

In [30]:
new_dict

{}

In [111]:
run_condition = True
while run_condition:
    new_dict = {}
    assigned = []
    for p in ppl_dict.keys():
        recips = []
        if 'exclude' in ppl_dict[p]:
            exclude = ppl_dict[p]['exclude']
            exclude = list(exclude.split(","))
            recips = list(set(ppl_dict.keys()) - set(exclude) - set(assigned))
        else:
            recips = list(set(ppl_dict.keys()) - set(assigned))
        
        try:
            recips.remove(p)
        except ValueError:
            pass

        try:
            rand = random.randint(0, (len(recips))-1)
            random_recip = recips[rand]
            assigned.append(random_recip)
            tmp_dict = {p: random_recip}
            new_dict.update(tmp_dict)
            del tmp_dict
            run_condition = False
        except ValueError:
            run_condition = True
            pass
new_dict

{'Person1': 'Person4',
 'Person2': 'Person1',
 'Person3': 'Person2',
 'Person4': 'Person3'}

In [74]:
new_dict

{'Person1': 'Person3', 'Person2': 'Person1', 'Person4': 'Person2'}

In [117]:
config_people = "config.people.yml"

In [118]:
def person_match(config_people: dict):
    '''doc-string'''
    # Read people config file
    ppl_dict = read_config(config_people)
    
    # Set run condition
    run_condition = True
    while run_condition:
        new_dict = {}
        assigned = []
        for p in ppl_dict.keys():
            recips = []
            if 'exclude' in ppl_dict[p]:
                exclude = ppl_dict[p]['exclude']
                exclude = list(exclude.split(","))
                recips = list(set(ppl_dict.keys()) - set(exclude) - set(assigned))
            else:
                recips = list(set(ppl_dict.keys()) - set(assigned))

            try:
                recips.remove(p)
            except ValueError:
                pass

            try:
                rand = random.randint(0, (len(recips))-1)
                random_recip = recips[rand]
                assigned.append(random_recip)
                tmp_dict = {p: random_recip}
                new_dict.update(tmp_dict)
                del tmp_dict
                run_condition = False
            except ValueError:
                run_condition = True
                pass
    return new_dict

In [120]:
sample = person_match(config_people)

In [126]:
sample

{'Person1': 'Person3',
 'Person2': 'Person4',
 'Person3': 'Person2',
 'Person4': 'Person1'}

In [122]:
ppl_dict

{'Person1': {'email': 'Person1@mail.com', 'exclude': 'Person2'},
 'Person2': {'email': 'Person2@mail.com'},
 'Person3': {'email': 'Person3@mail.com'},
 'Person4': {'email': 'Person4@mail.com'}}

In [129]:
for p in sample.keys():
    recipient = sample[p]
    sender = ppl_dict[p]
    print(recipient)

Person3
Person4
Person2
Person1


In [130]:
def gift_exchange(config_file, config_people, dry_run = False, verbose = False, budget = 15.00, year = 2019):
    '''
    Randomized gift exchange function. Randomizes list of people and
    matches one person to another.
    
    Args:
    config_file: configuration file that contains login credentials for (throwaway) G-mail account
    config_people: configuration file that contains names, and email addresses (and exclusion list)
    dry_run: boolean - if true, names and recipients are printed. If false, then emails are sent.
    verbose: boolean - if true, prints additional information to screen.
    budget: float - dollar amount for the budget
    year: int - year of the gift exchange
    
    Returns:
    None
    '''
    
    auto = read_config(cfg_file)
    ppl_dict = read_config(ppl_file)
    
    # Participants
    people = dict_key_to_list(ppl_dict)
    
    match_dict = person_match(config_people)
    
    # For each person, assemble a list of possible recipients.
    for p in ppl_dict.keys():
        random_recip = match_dict[p]
        if dry_run is True:
            print(f"{p} gives to {random_recip}")
        else:
            msg = f"Hi {p}, \n\n \
            You are giving to {random_recip}. \
            \n\n \
            The recommended budget is ${budget}, \
            but please feel free to spend whatever is appropriate. \
            \n\n \
            Take care. \
            \n\n \
            -- \
            \n\n \
            This is an automated email. \
            \n\n \
            Please, do not reply to this email."
            subject = f"Family Secret Santa Gift Exchange {year}"
            send_email(auto['email'],auto['password'],
                       ppl_dict[p]['email'],subject,msg)
            if verbose is True:
                print(f"Sent email to {p}")

In [131]:
gift_exchange("config.gift.yml",config_people,dry_run=True,year=2020)

Person1 gives to Person3
Person2 gives to Person4
Person3 gives to Person2
Person4 gives to Person1
